# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrects import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code == 200

True

In [4]:
data = data.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3003088647375,
 'week52high': 183.42,
 'week52low': 119.73,
 'week52highSplitAdjustOnly': 190.7,
 'week52lowSplitAdjustOnly': 117.56,
 'week52change': 0.4648583267887865,
 'sharesOutstanding': 17196328819,
 'float': 0,
 'avg10Volume': 121279287,
 'avg30Volume': 112925351,
 'day200MovingAvg': 150.51,
 'day50MovingAvg': 158.46,
 'employees': 148114,
 'ttmEPS': 11.46,
 'ttmDividendRate': 0.8759517731319095,
 'dividendYield': 0.005068304796636582,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-30',
 'nextEarningsDate': '2022-01-23',
 'peRatio': 15.23172923514323,
 'beta': 1.3258990027377833,
 'maxChangePercent': 67.92157995847961,
 'year5ChangePercent': 5.57481299412109,
 'year2ChangePercent': 1.6111009698579846,
 'year1ChangePercent': 0.4451413727169075,
 'ytdChangePercent': 0.3466802509936678,
 'month6ChangePercent': 0.38762693473119036,
 'month3ChangePercent': 0.18093578791531573,
 'month1ChangePercent': 0.17346489446155186,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.4451413727169075

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], index=my_columns), ignore_index=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,153.090,0.340029,N/A
1,AAL,17.010,0.004214,N/A
2,AAP,238.190,0.525297,N/A
3,AAPL,179.420,0.444863,N/A
4,ABBV,132.805,0.258085,N/A
...,...,...,...,...
500,YUM,136.460,0.280544,N/A
501,ZBH,126.670,-0.114307,N/A
502,ZBRA,613.200,0.610536,N/A
503,ZION,63.040,0.510925,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.660,2.315696,N/A
1,DVN,40.940,1.677838,N/A
2,FTNT,316.329,1.593489,N/A
3,MCHP,86.500,1.476878,N/A
4,F,20.890,1.286449,N/A
5,MRO,15.210,1.237043,N/A
6,FANG,105.360,1.188062,N/A
7,NVDA,293.270,1.180254,N/A
8,NUE,119.040,1.099035,N/A
9,IT,314.780,1.085277,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! \nPlease enter an integer: ")
        portfolio_input()
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000
10000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

C:\ProgramData\Anaconda3\envs\algorithm_trading\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.660,2.315696,2419
1,DVN,40.940,1.677838,4885
2,FTNT,316.329,1.593489,632
3,MCHP,86.500,1.476878,2312
4,F,20.890,1.286449,9573
5,MRO,15.210,1.237043,13149
6,FANG,105.360,1.188062,1898
7,NVDA,293.270,1.180254,681
8,NUE,119.040,1.099035,1680
9,IT,314.780,1.085277,635


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [29]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [30]:
for symbol_string in symbol_strings:
    batch_api_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ], index=hqm_columns), ignore_index=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.12,N/A,0.340935,N/A,0.095567,N/A,-0.094554,N/A,-0.028531,N/A,N/A
1,AAL,17.60,N/A,0.004115,N/A,-0.276936,N/A,-0.118836,N/A,-0.161953,N/A,N/A
2,AAP,238.69,N/A,0.529192,N/A,0.188819,N/A,0.180761,N/A,-0.021326,N/A,N/A
3,AAPL,180.58,N/A,0.450384,N/A,0.392439,N/A,0.183382,N/A,0.180236,N/A,N/A
4,ABBV,133.65,N/A,0.248134,N/A,0.123201,N/A,0.193851,N/A,0.082686,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.64,N/A,0.269598,N/A,0.116495,N/A,0.029026,N/A,0.060501,N/A,N/A
501,ZBH,124.61,N/A,-0.113674,N/A,-0.208518,N/A,-0.111487,N/A,-0.054165,N/A,N/A
502,ZBRA,603.78,N/A,0.603026,N/A,0.195987,N/A,0.043298,N/A,0.022869,N/A,N/A
503,ZION,62.65,N/A,0.511089,N/A,0.121855,N/A,0.056911,N/A,-0.06864,N/A,N/A


In [31]:
hqm_dataframe.fillna(0, inplace=True)

In [32]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']
for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_colomn = f'{time_period} Return Percentile'
        change_colomn = f'{time_period} Price Return'
        hqm_dataframe.loc[row, percentile_colomn] = stats.percentileofscore(hqm_dataframe[change_colomn], hqm_dataframe.loc[row, change_colomn])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.12,N/A,0.340935,0.621782,0.095567,0.639604,-0.094554,0.136634,-0.028531,0.427723,N/A
1,AAL,17.60,N/A,0.004115,0.164356,-0.276936,0.033663,-0.118836,0.09901,-0.161953,0.023762,N/A
2,AAP,238.69,N/A,0.529192,0.821782,0.188819,0.792079,0.180761,0.906931,-0.021326,0.465347,N/A
3,AAPL,180.58,N/A,0.450384,0.738614,0.392439,0.962376,0.183382,0.908911,0.180236,0.99604,N/A
4,ABBV,133.65,N/A,0.248134,0.485149,0.123201,0.69505,0.193851,0.922772,0.082686,0.940594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.64,N/A,0.269598,0.506931,0.116495,0.681188,0.029026,0.491089,0.060501,0.885149,N/A
501,ZBH,124.61,N/A,-0.113674,0.079208,-0.208518,0.079208,-0.111487,0.112871,-0.054165,0.281188,N/A
502,ZBRA,603.78,N/A,0.603026,0.887129,0.195987,0.805941,0.043298,0.540594,0.022869,0.70495,N/A
503,ZION,62.65,N/A,0.511089,0.811881,0.121855,0.693069,0.056911,0.6,-0.068640,0.215842,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [33]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.12,N/A,0.340935,0.621782,0.095567,0.639604,-0.094554,0.136634,-0.028531,0.427723,0.456436
1,AAL,17.60,N/A,0.004115,0.164356,-0.276936,0.033663,-0.118836,0.09901,-0.161953,0.023762,0.080198
2,AAP,238.69,N/A,0.529192,0.821782,0.188819,0.792079,0.180761,0.906931,-0.021326,0.465347,0.746535
3,AAPL,180.58,N/A,0.450384,0.738614,0.392439,0.962376,0.183382,0.908911,0.180236,0.99604,0.901485
4,ABBV,133.65,N/A,0.248134,0.485149,0.123201,0.69505,0.193851,0.922772,0.082686,0.940594,0.760891
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,136.64,N/A,0.269598,0.506931,0.116495,0.681188,0.029026,0.491089,0.060501,0.885149,0.641089
501,ZBH,124.61,N/A,-0.113674,0.079208,-0.208518,0.079208,-0.111487,0.112871,-0.054165,0.281188,0.138119
502,ZBRA,603.78,N/A,0.603026,0.887129,0.195987,0.805941,0.043298,0.540594,0.022869,0.70495,0.734653
503,ZION,62.65,N/A,0.511089,0.811881,0.121855,0.693069,0.056911,0.6,-0.068640,0.215842,0.580198


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [34]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.70,N/A,2.350221,1.0,0.833816,0.99802,0.222396,0.944554,0.080159,0.932673,0.968812
1,F,20.92,N/A,1.327265,0.992079,0.348785,0.948515,0.608277,0.99802,0.055213,0.863366,0.950495
2,LOW,258.01,N/A,0.667801,0.932673,0.375994,0.954455,0.264946,0.968317,0.089546,0.944554,0.95
3,INTU,652.81,N/A,0.872585,0.972277,0.431901,0.976238,0.206531,0.926733,0.066674,0.90495,0.94505
4,PLD,159.59,N/A,0.704832,0.950495,0.311810,0.932673,0.225114,0.946535,0.089990,0.946535,0.944059
5,AZO,2088.85,N/A,0.733890,0.958416,0.435022,0.978218,0.287095,0.978218,0.053387,0.861386,0.944059
6,DRE,64.43,N/A,0.684995,0.936634,0.287487,0.916832,0.249995,0.956436,0.098317,0.966337,0.944059
7,AVGO,626.83,N/A,0.585592,0.869307,0.348064,0.946535,0.262893,0.964356,0.106066,0.978218,0.939604
8,EXR,219.53,N/A,0.993125,0.980198,0.364882,0.952475,0.151920,0.877228,0.076971,0.926733,0.934158
9,KEYS,207.78,N/A,0.644787,0.920792,0.393459,0.964356,0.147443,0.867327,0.097046,0.964356,0.929208


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [35]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

C:\ProgramData\Anaconda3\envs\algorithm_trading\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.70,2389,2.350221,1.0,0.833816,0.99802,0.222396,0.944554,0.080159,0.932673,0.968812
1,F,20.92,9560,1.327265,0.992079,0.348785,0.948515,0.608277,0.99802,0.055213,0.863366,0.950495
2,LOW,258.01,775,0.667801,0.932673,0.375994,0.954455,0.264946,0.968317,0.089546,0.944554,0.95
3,INTU,652.81,306,0.872585,0.972277,0.431901,0.976238,0.206531,0.926733,0.066674,0.90495,0.94505
4,PLD,159.59,1253,0.704832,0.950495,0.311810,0.932673,0.225114,0.946535,0.089990,0.946535,0.944059
5,AZO,2088.85,95,0.733890,0.958416,0.435022,0.978218,0.287095,0.978218,0.053387,0.861386,0.944059
6,DRE,64.43,3104,0.684995,0.936634,0.287487,0.916832,0.249995,0.956436,0.098317,0.966337,0.944059
7,AVGO,626.83,319,0.585592,0.869307,0.348064,0.946535,0.262893,0.964356,0.106066,0.978218,0.939604
8,EXR,219.53,911,0.993125,0.980198,0.364882,0.952475,0.151920,0.877228,0.076971,0.926733,0.934158
9,KEYS,207.78,962,0.644787,0.920792,0.393459,0.964356,0.147443,0.867327,0.097046,0.964356,0.929208


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [36]:
writer = pd.ExcelWriter('momentum_stratagy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momemtum Stratagy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [37]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [38]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}
for column in column_formats.keys():
    writer.sheets['Momemtum Stratagy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momemtum Stratagy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [39]:
writer.save()